# FACE RECOGNITION BASE MODEL

## BASIS THEORY

In [1]:
import glob
import os

import cv2

from FPTvision.app import FaceAnalysis
from FPTvision.model_zoo import  arcface_onnx

## DETECTION

Detect On Camera

In [2]:
# Khởi tạo mô hình
model = FaceAnalysis()
model.prepare(ctx_id=0, det_size=(640, 480))

# Truy cập webcam
cap = cv2.VideoCapture(0)

# Thiết lập các thuộc tính cho webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Thiết lập chiều rộng khung hình
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Thiết lập chiều cao khung hình
cap.set(cv2.CAP_PROP_FPS, 60)  # Thiết lập tốc độ ghi hình (60 khung hình/giây)

while True:
    # Đọc khung hình từ webcam
    ret, frame = cap.read()
    
    # Lật ngược khung hình (phản chiếu qua trục dọc)
    frame = cv2.flip(frame, 1)

    # Dò tìm khuôn mặt
    faces = model.get(frame)

    for face in faces:
        # Hiển thị bouding box (đổi màu bounding box sang màu đỏ)
        bbox = face.bbox.astype(int)
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)
        
        # Hiển thị detection score
        cv2.putText(frame, f'Score: {face.det_score:.2f}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Hiển thị tọa độ landmark (Mắt, Mũi, Miệng)
        landmarks = face.kps
        landmark_indices = [0, 1, 2, 3, 4]
        for index in landmark_indices:
            x, y = landmarks[index]
            cv2.circle(frame, (int(x), int(y)), 2, (0, 255, 0), -1)
                    
    
    # Hiển thị kết quả trên cửa sổ OpenCV
    cv2.imshow('Face Detection', frame)

    # Nhấn 'q' để thoát và đóng camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Đóng camera và tắt tất cả cửa sổ OpenCV
cap.release()
cv2.destroyAllWindows()

c:\Users\nguye\anaconda3\envs\FPTForge\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
set det-size: (640, 480)


## RECOGNITION

Recognition on camera

In [1]:
import os
import glob
import cv2

from FPTvision.app import FaceAnalysis
from FPTvision.model_zoo import  arcface_onnx

# Khởi tạo mô hình
Detection = FaceAnalysis()
Detection.prepare(ctx_id=0, det_size=(640, 480))

Recognition = arcface_onnx.ArcFaceONNX()
Recognition.prepare(ctx_id=0)


# Tạo từ điển để lưu trữ embedding cho mỗi người
embeddings_dict = {}

aligned_face_dirs = glob.glob(os.path.join('FPTvision\\alinged\\', '*'))

for face_dir in aligned_face_dirs:
    # Lấy tên người từ tên thư mục
    person_name = os.path.basename(face_dir)

    face_files = glob.glob(os.path.join(face_dir, '*.jpg'))
    if len(face_files) > 0:
        face_images = [cv2.imread(file) for file in face_files]
        
        # Trích xuất embeddings cho mỗi bức ảnh
        face_embeddings = Recognition.get_feat(face_images)

        # Thêm vào từ điển
        embeddings_dict[person_name] = face_embeddings 

# Truy cập webcam
cap = cv2.VideoCapture(0)

# Thiết lập các thuộc tính cho webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Thiết lập chiều rộng khung hình
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Thiết lập chiều cao khung hình
cap.set(cv2.CAP_PROP_FPS, 60)  # Thiết lập tốc độ ghi hình (60 khung hình/giây)

while True:
    # Đọc khung hình từ webcam
    ret, frame = cap.read()
    
    # Lật ngược khung hình (phản chiếu qua trục dọc)
    frame = cv2.flip(frame, 1)

    # Dò tìm khuôn mặt
    faces = Detection.get(frame)

    for face in faces:
        
        # Hiển thị bouding box (đổi màu bounding box sang màu đỏ)
        bbox = face.bbox.astype(int)
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)
        
        # Hiển thị detection score
        cv2.putText(frame, f'Score: {face.det_score:.2f}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Hiển thị tọa độ landmark (Mắt, Mũi, Miệng)
        landmarks = face.kps
        landmark_indices = [0, 1, 2, 3, 4]
        for index in landmark_indices:
            x, y = landmarks[index]
            cv2.circle(frame, (int(x), int(y)), 2, (0, 255, 0), -1)
        
        # Lấy vector embeding từ khuôn mặt trong frame hình
        detected_embedding = Recognition.get(frame, face)
        
        #! Lấy ra tên khuôn mặt có xác suất cao nhất
        identify_name = ""
        max_similarity = 0.0
        for person_name, embeddings in embeddings_dict.items():
            similarity = 0.0
            for embedding in embeddings:
                similarity = max(similarity, Recognition.compute_sim(detected_embedding, embedding))
            if similarity > max_similarity:
                identify_name = person_name
                max_similarity = similarity
                
        # In tên khuôn mặt vào openCV
        cv2.putText(frame, identify_name, (bbox[2] - 10, bbox[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (242, 111, 33), 2)
        
    # Hiển thị kết quả trên cửa sổ OpenCV
    cv2.imshow('Face Detection', frame)

    # Nhấn 'q' để thoát và đóng camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Đóng camera và tắt tất cả cửa sổ OpenCV
cap.release()
cv2.destroyAllWindows()

c:\Users\nguye\anaconda3\envs\FPTForge\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
set det-size: (640, 480)
input mean and std: 127.5 127.5
['FPTvision\\alinged\\SE171408', 'FPTvision\\alinged\\SE172383']


# Convert Embeding Vector

In [3]:
import glob
import os
import cv2
import json

Recognition = arcface_onnx.ArcFaceONNX()
Recognition.prepare(ctx_id=0)

# Tạo một từ điển để lưu trữ embedding cho mỗi người
embeddings_dict = {}

aligned_face_dirs = glob.glob(os.path.join('FPTvision\\alinged\\', '*'))
print(aligned_face_dirs)

for face_dir in aligned_face_dirs:
    # Lấy tên người từ tên thư mục
    person_name = os.path.basename(face_dir)

    face_files = glob.glob(os.path.join(face_dir, '*.jpg'))
    if len(face_files) > 0:
        face_images = [cv2.imread(file) for file in face_files]

        # Trích xuất embeddings cho mỗi bức ảnh
        face_embeddings = Recognition.get_feat(face_images)

        # Thêm vào từ điển theo cấu trúc mong muốn
        person_data = {}
        for i, embedding in enumerate(face_embeddings):
            vector_name = f'vector {i + 1}'
            person_data[vector_name] = embedding.tolist()
        
        embeddings_dict[person_name] = person_data

# Lưu từ điển vào tệp JSON
with open('embeddings.json', 'w') as json_file:
    json.dump(embeddings_dict, json_file)


input mean and std: 127.5 127.5
['FPTvision\\alinged\\SE171408', 'FPTvision\\alinged\\SE172383']


## Face Recognition With JSON

In [1]:
import os
import glob
import cv2
import json
import numpy as np

from FPTvision.app import FaceAnalysis
from FPTvision.model_zoo import arcface_onnx

# Khởi tạo mô hình
Detection = FaceAnalysis()
Detection.prepare(ctx_id=0, det_size=(640, 480))

Recognition = arcface_onnx.ArcFaceONNX(model_file='./FPTvision/models/Recognition/model.onnx')
Recognition.prepare(ctx_id=0)

# Đọc tệp JSON chứa vector embedding
with open('embeddings.json', 'r') as json_file:
    embeddings_dict = json.load(json_file)

# Truy cập webcam
cap = cv2.VideoCapture(0)

# Thiết lập các thuộc tính cho webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Thiết lập chiều rộng khung hình
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Thiết lập chiều cao khung hình
cap.set(cv2.CAP_PROP_FPS, 60)  # Thiết lập tốc độ ghi hình (60 khung hình/giây)

# Đặt trọng số cho mỗi vector embedding
WEIGHTS = [0.2, 0.2, 0.2, 0.2, 0.2]  # Có thể thay đổi giá trị này

while True:
    # Đọc khung hình từ webcam
    ret, frame = cap.read()
    
    # Lật ngược khung hình (phản chiếu qua trục dọc)
    frame = cv2.flip(frame, 1)

    # Dò tìm khuôn mặt
    faces = Detection.get(frame)

    for face in faces:
        # Hiển thị bounding box (đổi màu bounding box sang màu đỏ)
        bbox = face.bbox.astype(int)
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)
        
        # Hiển thị detection score
        cv2.putText(frame, f'Score: {face.det_score:.2f}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Hiển thị tọa độ landmark (Mắt, Mũi, Miệng)
        landmarks = face.kps
        landmark_indices = [0, 1, 2, 3, 4]
        for index in landmark_indices:
            x, y = landmarks[index]
            cv2.circle(frame, (int(x), int(y)), 2, (0, 255, 0), -1)
        
        # Lấy vector embedding từ khuôn mặt trong frame hình
        detected_embedding = Recognition.get(frame, face)

        # Lấy ra tên khuôn mặt có xác suất cao nhất
        identify_name = ""
        max_similarity = 0.0

        for person_name, embeddings in embeddings_dict.items():
            weighted_similarity = 0.0
            for i, (vector_name, vector) in enumerate(embeddings.items()):
                similarity = Recognition.compute_sim(np.array(vector), detected_embedding)
                weighted_similarity += WEIGHTS[i] * similarity
            
            if weighted_similarity > max_similarity:
                identify_name = person_name
                max_similarity = weighted_similarity

        # In tên khuôn mặt vào OpenCV
        cv2.putText(frame, identify_name, (bbox[2] - 10, bbox[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (242, 111, 33), 2)

    # Hiển thị kết quả trên cửa sổ OpenCV
    cv2.imshow('Face Detection', frame)

    # Nhấn 'q' để thoát và đóng camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Đóng camera và tắt tất cả cửa sổ OpenCV
cap.release()
cv2.destroyAllWindows()

c:\Users\nguye\anaconda3\envs\FPTForge\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
set det-size: (640, 480)
input mean and std: 127.5 127.5
